# Аналіз ефективності логістичних маршрутів

# Завантаження та попередня обробка даних

Завантажено дані з Excel-файлу `Loads 2024.xlsx`, аркуш `DATA`.
Виконано очищення: видалено записи з порожніми значеннями в ключових полях (`Rate`, `Miles`, `RPM`, `start`, `end`).
Числові поля переведено у відповідний тип, а дати — у формат `datetime`.

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_excel("../Loads 2024.xlsx", sheet_name="DATA")

df = df.dropna(subset=["Rate", "Miles", "RPM", "start", "end"])
df["Miles"] = pd.to_numeric(df["Miles"], errors="coerce")
df["Rate"] = pd.to_numeric(df["Rate"], errors="coerce")
df["RPM"] = pd.to_numeric(df["RPM"], errors="coerce")
df["start"] = pd.to_datetime(df["start"], errors="coerce")
df["end"] = pd.to_datetime(df["end"], errors="coerce")

# Розрахунок тривалості поїздки та середньої швидкості (MPH)

На основі полів `start` і `end` обчислено тривалість кожного рейсу в годинах.
Для валідних записів (де `Miles` та `Hours` > 0) обраховано середню швидкість у милях на годину (`MPH`).

In [3]:
df["Hours"] = (df["end"] - df["start"]).dt.total_seconds() / 3600
df = df[(df["Hours"] > 0) & (df["Miles"] > 0)]
df["MPH"] = df["Miles"] / df["Hours"]

# Побудова унікального ідентифікатора маршруту

Згенеровано поле `Route`, яке поєднує назви міст і штатів відправлення та призначення у форматі:
`From City, From State → To City, To State`

In [4]:
df["Route"] = (
    df["From City"].str.strip() + ", " +
    df["From State"].str.strip() + " → " +
    df["To City"].str.strip() + ", " +
    df["To State"].str.strip()
)

# Групування та обчислення середніх показників по маршрутах

Дані агреговано за маршрутом. Для кожного маршруту розраховано:
- Середній `MPH`
- Середній `RPM` (дохід за милю)
- Середню кількість миль (`Miles`)
- Середній тариф (`Rate`)
- Кількість поїздок (`Trips`)

In [5]:
grouped = df.groupby("Route").agg({
    "MPH": "mean",
    "RPM": "mean",
    "Miles": "mean",
    "Rate": "mean",
    "start": "count"
}).rename(columns={"start": "Trips"}).reset_index()

# Масштабування метрик до єдиного діапазону

Для забезпечення порівнюваності показників виконано нормалізацію `MPH` та `RPM`
до діапазону [0, 1] за допомогою `MinMaxScaler`.

In [6]:
scaler = MinMaxScaler()
grouped[["MPH_norm", "RPM_norm"]] = scaler.fit_transform(grouped[["MPH", "RPM"]])

# Розрахунок зваженої ефективності маршруту

На основі нормалізованих метрик розраховано оцінку `Score`:
- 60% — швидкість (`MPH`)
- 40% — дохід за милю (`RPM`)

Після чого скор масштабовано до шкали від 1 до 10 (`Score_1_10`), де 10 — найефективніший маршрут.

In [7]:
grouped["Score"] = grouped["MPH_norm"] * 0.6 + grouped["RPM_norm"] * 0.4
grouped["Score_1_10"] = (grouped["Score"] * 9 + 1).round(2)

# Формування фінальної таблиці з ефективністю маршрутів

Побудовано підсумкову таблицю з ключовими метриками та оцінкою ефективності для кожного маршруту.
Таблиця відсортована за зменшенням оцінки (`Score_1_10`) для подальшого аналізу.

In [8]:
result = grouped[[
    "Route", "MPH", "RPM", "Miles", "Rate", "Trips", "Score_1_10"
]].sort_values("Score_1_10", ascending=False).reset_index(drop=True)

result

,Route,MPH,RPM,Miles,Rate,Trips,Score_1_10
0,"AMERICAN CANYON, CA → ARLINGTON HEIGHTS, IL",62.736111,0.088600,9034.000000,800.000000,1,6.40
1,"STOCKTON, CA → STOCKTON, CA",0.013889,600.000000,2.000000,1200.000000,1,4.60
2,"ALAMEDA, CA → VAN BUREN TWP, MI",29.742424,0.998682,4282.909091,2495.454545,11,3.57
3,"PATTERSON, CA → LIVONIA, TX",29.423611,2.088700,4237.000000,8850.000000,1,3.54
4,"ELGIN, IL → MANKATO, MN",27.784722,0.223700,4001.000000,895.000000,1,3.39
...,...,...,...,...,...,...,...
4793,"SALEM, SC → GREENVILLE, SC",0.368056,6.792500,53.000000,360.000000,1,1.07
4794,"MODESTO, CA → ACAMPO, CA",0.375000,6.481500,54.000000,350.000000,1,1.07
4795,"CHICAGO RIDGE, IL → ELWOOD, IL",0.395833,5.263200,57.000000,300.000000,1,1.06
4796,"RIVERSIDE, CA → VERNON, CA",0.402778,5.172400,58.000000,300.000000,1,1.06
